<a href="https://colab.research.google.com/github/2019ht12169/Thesis/blob/main/parse_and_clean_data_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import all required libraries
!pip install better_profanity
!pip install emoji
from bs4 import BeautifulSoup
import re
from better_profanity import profanity
import pandas as pd
import csv
import emoji

In [ ]:
# using google drive to upload dataset related files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#from google.colab import files [ contingency to upload dataset files in case of drive connection failure]
#upload=files.upload()

In [32]:
def is_custom_URL_pattern_matched(text):
  return bool(text.find('http')!=-1) and bool(re.match(r"https?:\s*//\S+|www\.\s*\S+|[a-z]+\.\s*\S+", text))

In [33]:
# checking for pattern and http before removing url
def remove_urls(text):
    url_pattern = re.compile(r'https?:\s*//\S+|www\.\s*\S+|[a-z]+\.\s*\S+')
    #url_pattern = re.compile(r'https?://\S+|www\.\S+')
    if is_custom_URL_pattern_matched(text):
      url = url_pattern.sub(r'', text)
    else:
      url=text
    return url

In [34]:
print(remove_urls('http://www.amazon.com/gp/product/b00ppag8lk?'))
print(remove_urls('Excellent. Excellent'))


Excellent. Excellent


In [35]:
# replace all emoji's in dataset with equivalent text
def replaceEmojis(text):
  replacedText = emoji.demojize(text, delimiters=(' ',' '))
  return replacedText

In [37]:
print(replaceEmojis("hello hi👍 hi😊"))

hello hi thumbs_up  hi smiling_face_with_smiling_eyes 


In [40]:
# removing single char reviews if any to avoid noise
def is_single_char_sentence(text):
  return len(text)==1 and bool(re.match(r"\w", text))

In [39]:
print(is_single_char_sentence('a'))

True


In [42]:
# converting chat word terminology to full text
chat_words_str = """
AFAIK=As Far As I Know
ASAP=As Soon As Possible
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BRT=Be Right There
BTW=By The Way
B4=Before
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My Ass Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
OIC=Oh I See
PITA=Pain In The Ass
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My Ass Off
SK8=Skate
ASL=Age, Sex, Location
THX=Thank You
U=You
U2=You Too
WB=Welcome Back
WTF=What The Fuck
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [41]:
#convert emoticons to equivalent text
EMOTICONS = {
    u":‑\)":"Happy face",
    u":\)":"Happy face",
    u":-\]":"Happy face",
    u":\]":"Happy face",
    u":-3":"Happy face",
    u":3":"Happy face",
    u":->":"Happy face",
    u":>":"Happy face",
    u"8-\)":"Happy face",
    u":o\)":"Happy face",
    u":-\}":"Happy face",
    u":\}":"Happy face",
    u":-\)":"Happy face",
    u":c\)":"Happy face",
    u":\^\)":"Happy face",
    u"=\]":"Happy face",
    u"=\)":"Happy face",
    u":‑D":"Laughing",
    u":D":"Laughing",
    u"8‑D":"Laughing",
    u"8D":"Laughing",
    u"X‑D":"Laughing",
    u"XD":"Laughing",
    u"=D":"Laughing",
    u"=3":"Laughing",
    u"B\^D":"Laughing",
    u":-\)\)":"Very happy",
    u":‑\(":"angry",
    u":-\(":"angry",
    u":\(":"angry",
    u":‑c":"angry",
    u":c":"angry",
    u":‑<":"angry",
    u":<":"angry",
    u":‑\[":"angry",
    u":\[":"angry",
    u":-\|\|":"angry",
    u">:\[":"angry",
    u":\{":"angry",
    u":@":"angry",
    u">:\(":"angry",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"smirk",
    u";\)":"smirk",
    u"\*-\)":"smirk",
    u"\*\)":"smirk",
    u";‑\]":"smirk",
    u";\]":"smirk",
    u";\^\)":"smirk",
    u":‑,":"smirk",
    u";D":"smirk",
    u":‑P":"Tongue sticking out",
    u":P":"Tongue sticking out",
    u"X‑P":"Tongue sticking out",
    u"XP":"Tongue sticking out",
    u":‑Þ":"Tongue sticking out",
    u":Þ":"Tongue sticking out",
    u":b":"Tongue sticking out",
    u"d:":"Tongue sticking out",
    u"=p":"Tongue sticking out",
    u">:P":"Tongue sticking out",
    u":‑/":"Skeptical",
    u":/":"Skeptical",
    u":-[.]":"Skeptical",
    u">:[(\\\)]":"Skeptical",
    u">:/":"Skeptical",
    u":[(\\\)]":"Skeptical",
    u"=/":"Skeptical",
    u"=[(\\\)]":"Skeptical",
    u":L":"Skeptical",
    u"=L":"Skeptical",
    u":S":"Skeptical",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed",
    u":‑x":"Sealed lips",
    u":x":"Sealed lips",
    u":‑#":"Sealed lips",
    u":#":"Sealed lips",
    u":‑&":"Sealed lips",
    u":&":"Sealed lips",
    u"O:‑\)":"innocent",
    u"O:\)":"innocent",
    u"0:‑3":"innocent",
    u"0:3":"innocent",
    u"0:‑\)":"innocent",
    u"0:\)":"innocent",
    u":‑b":"Tongue sticking out",
    u"0;\^\)":"innocent",
    u">:‑\)":"devilish",
    u">:\)":"devilish",
    u"\}:‑\)":"devilish",
    u"\}:\)":"devilish",
    u"3:‑\)":"devilish",
    u"3:\)":"devilish",
    u">;\)":"devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"confused",
    u"%\)":"confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Embarrassed",
    u"\(\^_\^;\)":"Embarrassed",
    u"\(-_-;\)":"Embarrassed",
    u"\(~_~;\) \(・\.・;\)":"Embarrassed",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"sign of respect",
    u"_\(\._\.\)_":"sign of respect",
    u"<\(_ _\)>":"sign of respect",
    u"<m\(__\)m>":"sign of respect",
    u"m\(__\)m":"sign of respect",
    u"m\(_ _\)m":"sign of respect",
    u"\('_'\)":"Crying",
    u"\(/_;\)":"Crying",
    u"\(T_T\) \(;_;\)":"Crying",
    u"\(;_;":"Crying",
    u"\(;_:\)":"Crying",
    u"\(;O;\)":"Crying",
    u"\(:_;\)":"Crying",
    u"\(ToT\)":"Crying",
    u";_;":"Crying",
    u";-;":"Crying",
    u";n;":"Crying",
    u";;":"Crying",
    u"Q\.Q":"Crying",
    u"T\.T":"Crying",
    u"QQ":"Crying",
    u"Q_Q":"Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Cheerful",
    u"\(\^_\^\)v":"Cheerful",
    u"\(\(d[-_-]b\)\)":"Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Deflated"
}
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', ' '.join(EMOTICONS[emot].replace(",","").split()), text)
    return text

In [11]:
# stop word removal [our use case is sentiment analysis so, keeping negation words intact]
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_default = stopwords.words('english')
stopwords_toBeRemoved=['no', 'nor', 'not','but','don',"don't","aren't",'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
for word in stopwords_toBeRemoved:
  stopwords_default.remove(word)
STOPWORDS = set(stopwords_default)
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
# nullify profane words in dataset
def filter_profanity(sentence): 
   profanity.load_censor_words()
   if profanity.contains_profanity(sentence):
    sentence = profanity.censor(sentence)
   return sentence

In [43]:
#various possible steps for preprocessing
def preprocessText(reviewText):
      # remove url's
      reviewText = remove_urls(reviewText)
      # remove html tags
      soup = BeautifulSoup(reviewText)
      reviewText = soup.get_text()
      # convert acronyms/chat terminology to full text
      reviewText = chat_words_conversion(reviewText)
      #convert emoticons to text
      reviewText = convert_emoticons(reviewText)
      #convert emoji to text
      reviewText = replaceEmojis(reviewText)
      # replace these tags and quote,comma to avoid parsing conflicts
      reviewText = reviewText.replace('"','')
      reviewText = reviewText.replace('<br>','')
      reviewText = reviewText.replace('<br />','')
      reviewText = reviewText.replace('/>','')
      reviewText = re.sub(',', '', reviewText)
      # remove stop words
      #reviewText = remove_stopwords(reviewText)
      # remove new line characters
      #reviewText = reviewText.replace('\n',' ')
      # remove punctuation 
      #reviewText = re.sub(r'[^\w\s]', '', reviewText)
      # do censoring
      #reviewText = profanity.censor(reviewText)
      #remove astricks to nullify censored words
      #reviewText = reviewText.replace('*','')
      # remove single char sentences
      #reviewText = remove_single_char_sentences(reviewText)
      # convert all text to lower case
      reviewText = reviewText.lower()
      return reviewText

In [ ]:
#download data file from drive
tsv_file='/content/drive/MyDrive/projects/data/amazon_reviews_Electronics.tsv'
csv_table=pd.read_table(tsv_file,sep='\t',error_bad_lines=False)
# convert data file into csv format
csv_table.to_csv('/content/drive/MyDrive/projects/data/reviews_Electronics.csv',index=False,header=True)


In [15]:
#making data frame out of the csv version of data file
data = pd.read_csv('/content/drive/MyDrive/projects/data/reviews_Electronics.csv')
# drop unneeded columns and print it
df = data.drop(['marketplace','customer_id','review_id','product_id',
       'product_parent','product_title', 'product_category','helpful_votes', 'total_votes', 'vine', 'verified_purchase'], axis='columns')
df.head()

,star_rating,review_headline,review_body,review_date
0,5,Five Stars,As described.,2015-08-31
1,5,It works as advertising.,It works as advertising.,2015-08-31
2,5,Five Stars,Works pissa,2015-08-31
3,1,One Star,Did not work at all.,2015-08-31
4,5,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [16]:
#create new attributes as per need 
#labeling data based on star rating and combining review headlines and body
df.loc[df['star_rating'] < 4, 'label'] = '0' 
df.loc[df['star_rating'] >= 4, 'label'] = '1' 
df=df.assign(text=lambda x: x['review_headline']+' '+x['review_body'])

In [18]:
# creating fresh df with needed features
import numpy as np
df_map_result = pd.DataFrame({'label': df['label'],
    'text': df['text'],
    'review_date': df['review_date']})
df_map_result = df_map_result.astype({'label':int,
'text':str,
'review_date':np.datetime64})

df_map_result.to_csv("/content/drive/MyDrive/projects/data/dataFile_reviews_Electronics.csv",index=False,header=True)
#df_map_result.sample(10001).to_csv("/content/drive/MyDrive/projects/data/dataFile_reviews_Electronics_small.csv",index=False,header=True)

In [12]:
# utility method for writing headers to data set
def writeHeadersToFile(file):
  file.write('"label"')
  file.write(",")
  file.write('"review_date"')
  file.write(",")
  file.write('"text"')
  file.write("\n")

In [13]:
# utility method for writing data rows to data set
def writeDataToFile(file,row):
  file.write('"{}"'.format(row[0]))
  file.write(",")
  file.write('"{}"'.format(row[2]))
  file.write(",")
  file.write('"{}"'.format(preprocessText(row[1])))
  file.write("\n")

In [22]:
# separating positive and negative reviews for understanding data better 
# and balancing train data set
file1 = open("/content/drive/MyDrive/projects/data/dataFile_latest_pos.csv","w")
file2 = open("/content/drive/MyDrive/projects/data/dataFile_latest_neg.csv","w")
writeHeadersToFile(file1)
writeHeadersToFile(file2)
poscount=0
negcount=0
with open('/content/drive/MyDrive/projects/data/dataFile_reviews_Electronics.csv', 'r') as file:
    reader = csv.reader(file, delimiter=',')
    next(reader, None) 
    for row in reader:
      if int(row[0]) == 0:
        negcount = negcount+1
        writeDataToFile(file2,row)
      else:
        poscount = poscount+1
        writeDataToFile(file1,row)     
print(negcount)
print(poscount)

775236
2315788


In [45]:
# balance higher class labels with lower class labels 
# by extracting subset of higher class
actualPos = pd.read_csv('/content/drive/MyDrive/projects/data/dataFile_latest_pos.csv')
#actualPos.sample(negcount).to_csv("/content/drive/MyDrive/projects/data/dataFile_latest_pos.csv",index=False,header=True)
actualPos.sample(775236).to_csv("/content/drive/MyDrive/projects/data/dataFile_latest_pos.csv",index=False,header=True)
actualPos = pd.read_csv('/content/drive/MyDrive/projects/data/dataFile_latest_pos.csv')
actualPos.head()
len(actualPos)

775236

In [46]:
  # creating train and test dataset files after all preprocessing
  file_1 = open("/content/drive/MyDrive/projects/data/dataFile_latest_test.csv","w")
  file_2 = open("/content/drive/MyDrive/projects/data/dataFile_latest_train.csv","w")
  writeHeadersToFile(file_1)
  writeHeadersToFile(file_2)

In [47]:
#creating train and test datasets as combination of positive and negative reviews
count = 0
with open('/content/drive/MyDrive/projects/data/dataFile_latest_pos.csv', 'r') as file:
  reader = csv.reader(file, delimiter=',')
  next(reader, None) 
  for row in reader:
    count = count+1
    if (count%5) == 0:
      file_1.write('"{}"'.format(row[0]))
      file_1.write(",")
      file_1.write('"{}"'.format(row[1]))
      file_1.write(",")
      file_1.write('"{}"'.format(row[2]))
      file_1.write("\n")
    else:
      file_2.write('"{}"'.format(row[0]))
      file_2.write(",")
      file_2.write('"{}"'.format(row[1]))
      file_2.write(",")
      file_2.write('"{}"'.format(row[2]))
      file_2.write("\n")

In [ ]:
#creating train and test datasets as combination of positive and negative reviews
count = 0
with open('/content/drive/MyDrive/projects/data/dataFile_latest_neg.csv', 'r') as file:
  reader = csv.reader(file, delimiter=',')
  next(reader, None) 
  for row in reader:
    count = count+1
    if count%5 == 0:
      file_1.write('"{}"'.format(row[0]))
      file_1.write(",")
      file_1.write('"{}"'.format(row[1]))
      file_1.write(",")
      file_1.write('"{}"'.format(row[2]))
      file_1.write("\n")
    else:
      file_2.write('"{}"'.format(row[0]))
      file_2.write(",")
      file_2.write('"{}"'.format(row[1]))
      file_2.write(",")
      file_2.write('"{}"'.format(row[2]))
      file_2.write("\n")